# Model comparison GPT 3.5 and GPT 4

Use Q&A from a topic (in this example, sociology, University level, Germany) to compare the answers from two Open AI models.
We use human answers as reference.

Authors: Andre Moreira (with the support of Sofia Moreira), August - Ocober, 2023

## Preamble

In [1]:
import pandas as pd
import numpy as np
import openai

import csv

from tqdm import tqdm
from time import sleep

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Upload the OpenAI keys

# add the name of the file that contains your Open AI keys information if you want to run the code in Part 2
file_keys = #
df_keys = pd.read_csv(file_keys)

openai.organization = df_keys.iloc[0,0]
openai.api_key = df_keys.iloc[0,1]

SyntaxError: invalid syntax (1623991479.py, line 4)

## Upload questions, connect with chatGPT and get answers

In [4]:
# Upload the questions (NB: this was prepared separately)

filename = "Cleaned_up_Qs.csv"
df = pd.read_csv(filename)
df.head()

,Number,Question
0,1,Eine Studentin bekommt eine Klausur mit einer ...
1,2,Nenne zwei strukturelle und zwei funktionelle ...
2,3,Nenne drei der sechs Bedürfnisse der Bedürfnis...
3,4,Nennen sie vier der sechs Entwicklungsbereiche...
4,5,Nennen sie vier der sechs Thesen aus dem Schne...


In [5]:
df[['Answer']] = "" # Create an answer column
max_Qs = df.shape[0] # Get the length of the dataframe 
print(df.shape, max_Qs)

(355, 3) 355


In [6]:
df.head()

,Number,Question,Answer
0,1,Eine Studentin bekommt eine Klausur mit einer ...,
1,2,Nenne zwei strukturelle und zwei funktionelle ...,
2,3,Nenne drei der sechs Bedürfnisse der Bedürfnis...,
3,4,Nennen sie vier der sechs Entwicklungsbereiche...,
4,5,Nennen sie vier der sechs Thesen aus dem Schne...,


In [9]:
# Version 1
# one-shot inference + exception handling when OpenAI "needs a break"

prompt_system = "You are a helpful assistant. You are an expert in medicine, psychology, sociology."

prompt_intro = "You are a helpful assistant. You are an expert in medicine, psychology, sociology. \
The following text contains a question in German. \
You will read it carefully and provide the right answer, in a short and concise language. \
Read each question carefully, take your time to chose the right asnwer. Do not halucinate.\
Do not invent things you do not know. If you do not know the answer, simply say 'no answer'. \
If the a question is a multiple choice, you answer with your choice and a short reasoning why.\
Please provide your answers in German."

prompt_Q_ex1 = "Was trifft auf das Big-Five Modell zu ? \
A)Beschreibt Traits / Gefühle in einer bestimmten Situation. \
B)Beschreibt Persönlichkeit ausschließlich bei Kindern und Jugendlichen \
C)Beruht auf einem lexikalischen Ansatz.\
D)Steht für Offenheit, Gewissenhaftigkeit, Extraversion, Vertrauen, Neurozitismus \
E)Bei Abweichungen liegt laut DSM-5 eine Persönlichkeitsstörung vor."

prompt_A_ex1 = "C) Beruht auf einem lexikalischen Ansatz. \
Das Big-Five-Modell ist ein Persönlichkeitsmodell, das auf dem lexikalischen Ansatz beruht. \
Dies bedeutet, dass die Persönlichkeitseigenschaften, die in diesem Modell beschrieben werden, \
aus der Sprache abgeleitet wurden. Es handelt sich um fünf grundlegende Dimensionen der \
Persönlichkeit: Neurotizismus, Extraversion, Offenheit für Erfahrungen, Verträglichkeit \
und Gewissenhaftigkeit."

a = 0
b = 50 # subset of Qs that we will use to compare with the human answers, available in a separate file for the first 50 Qs

str_error = ""

for i in tqdm(range(a, b)):
    try:
        prompt_Q = df.iloc[i,1]
        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",  # use a specific model (comment, uncomment for selection)
        #model="gpt-4-0613",
        messages=[
            {"role": "system", "content": prompt_system}, # re-inforce this message
            {"role": "user", "content": prompt_intro},
            {"role": "user", "content": prompt_Q_ex1},
            {"role": "assistant", "content": prompt_A_ex1},
            {"role": "user", "content": prompt_Q},
            ])
        
        df.iloc[i,2] = response['choices'][0]['message']['content']
        str_error = ""
    except Exception as str_err:
        print(str_err)
        str_error = "Taking a break. Will try again in 5 seconds"
        print("paused at", i)
        pass # site congested, do nothing
    
    if str_error != "":
        print(str_error)
        sleep(5) # wait for 5 seconds before trying again
        str_error = ""
        i = i - 1 # repeat where we stopped
        print("Resuming...")


100%|███████████████████████████████████████████| 50/50 [03:35<00:00,  4.30s/it]


NB:

GPT 3.5
100%|███████████████████████████████████████████| 50/50 [03:35<00:00,  4.30s/it]

GPT 4
100%|███████████████████████████████████████████| 40/40 [07:23<00:00, 11.09s/it]


In [10]:
pd.set_option('display.max_rows', None)
df.head(51)

,Number,Question,Answer
0,1,Eine Studentin bekommt eine Klausur mit einer ...,"C) Internal, global, spezifisch. Der Attributi..."
1,2,Nenne zwei strukturelle und zwei funktionelle ...,Zwei strukturelle Bildgebungsverfahren sind di...
2,3,Nenne drei der sechs Bedürfnisse der Bedürfnis...,Die Bedürfnispyramide nach Maslow umfasst die ...
3,4,Nennen sie vier der sechs Entwicklungsbereiche...,Die vier der sechs Entwicklungsbereiche der fr...
4,5,Nennen sie vier der sechs Thesen aus dem Schne...,"Entschuldigung, aber ich bin nicht vertraut mi..."
5,6,Was trifft auf das Big-Five Modell zu ? A)Besc...,C) Beruht auf einem lexikalischen Ansatz. Das ...
6,7,Das Biopsychovulnerable Modell besagt: A)Je h...,"A) Je höher die Vulnerabilität, desto weniger ..."
7,8,Welche Reihenfolge der Gedächtnisse hinsichtli...,C) Sensorisches Gedächtnis - Arbeitsgedächtnis...
8,9,Welche Aussage zum EEG trifft NICHT zu? A)Das ...,C) Die Frequenz beschreibt die Ausschläge der ...
9,10,zu ROC Kurven und Area under the Curve bei die...,"B) Eine aussagekräftige AUC erhält man, wenn m..."


In [13]:
# Save the results (un-/comment as needed)
df.to_csv('Cleaned_up_Q_and_A_QC_TEST-GPT3p5.csv', index=False)

# df.to_csv('Cleaned_up_Q_and_A_QC_TEST-GPT4.csv', index=False)


## Compare the models and summarize the results

### Note that this part assumes that the files for GPT 3.5 and GPT 4 have been generated in parts 1 and 2 above

In [7]:
# Q and A files (GPT 3.5, GPT 4 and Human responses)

filename = 'Cleaned_up_Q_and_A_QC_TEST-GPT3p5.csv'
df_gpt_3p5 = pd.read_csv(filename)

filename = 'Cleaned_up_Q_and_A_QC_TEST-GPT4.csv'
df_gpt_4 = pd.read_csv(filename)

filename = 'Cleaned_up_Q_and_A_Human_pub.csv' # this was prepared separately, manually
df_HH = pd.read_csv(filename)

In [8]:
# create a mask that marks the multiple-choice answers only - we will use them as a first benchmark 
lst = df_HH["Human FB (KM)"]
msk = []
for i in lst:
    if len(i)>1:
        msk = msk + [0]
    else:
        msk = msk + [1]
# convert to boolean values as np array to filter the dataframes
msk = np.array(msk, dtype = bool)

In [9]:
msk

array([ True, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True])

In [10]:
len(msk)

50

In [11]:
# Extract the answers with multiple choice only
df2_gpt_4 = df_gpt_4.head(len(msk))[msk]
df2_gpt_3p5 = df_gpt_3p5.head(len(msk))[msk]
df2_HH = df_HH[msk]

In [12]:
# Check how the human Q&A DF looks like 
df2_HH.head()

,Number,Question,Human FB (KM)
0,1,Eine Studentin bekommt eine Klausur mit einer ...,A
5,6,Was trifft auf das Big-Five Modell zu ? A)Besc...,C
6,7,Das Biopsychovulnerable Modell besagt: A)Je h...,A
7,8,Welche Reihenfolge der Gedächtnisse hinsichtli...,C
8,9,Welche Aussage zum EEG trifft NICHT zu? A)Das ...,C


In [13]:
# Check how the e.g. GPT 4 Q&A DF looks like 
df2_gpt_4.head()

,Number,Question,Answer
0,1,Eine Studentin bekommt eine Klausur mit einer ...,"A) External, variabel, spezifisch. Die Student..."
5,6,Was trifft auf das Big-Five Modell zu ? A)Besc...,C) Beruht auf einem lexikalischen Ansatz und D...
6,7,Das Biopsychovulnerable Modell besagt: A)Je h...,"A) Je höher die Vulnerabilität, desto weniger ..."
7,8,Welche Reihenfolge der Gedächtnisse hinsichtli...,C) Sensorisches Gedächtnis- Arbeitsgedächtnis-...
8,9,Welche Aussage zum EEG trifft NICHT zu? A)Das ...,C) Die Frequenz beschreibt die Ausschläge der ...


In [14]:
# Extract the choices
list_gpt_4 = df2_gpt_4.loc[:,'Answer'].str[:1].to_list()
list_gpt_3p5 = df2_gpt_3p5.loc[:,'Answer'].str[:1].to_list()
list_HH = df2_HH.loc[:,'Human FB (KM)'].str[:1].to_list()

In [15]:
# Create a DF that puts all answers side-by-side
df_comp = pd.DataFrame(list(zip(list_HH, list_gpt_3p5, list_gpt_4)), columns = ["Human", "GPT 3.5", "GPT 4"])

In [16]:
df_comp.head()

,Human,GPT 3.5,GPT 4
0,A,C,A
1,C,C,C
2,A,A,A
3,C,C,C
4,C,C,C


In [17]:
# Prepare a column comparing human and GPT 3.5 responses
df_comp["GPT 3.5_vs_H"] = df_comp.apply(lambda x: 1 if x["Human"] ==
                     x["GPT 3.5"] else 0, axis=1)

In [18]:
# Prepare a column comparing human and GPT 4 responses
df_comp["GPT 4_vs_H"] = df_comp.apply(lambda x: 1 if x["Human"] ==
                     x["GPT 4"] else 0, axis=1)

In [19]:
# Of the responses where both 3.5 and 4 disagree with the human response, which ones both models agree with each other?
df_comp["FF_agree"] = df_comp.apply(lambda x: 1 if (x["GPT 3.5_vs_H"] == 0 and
                                    x["GPT 4_vs_H"] == 0) and x["GPT 3.5"] == x["GPT 4"]
                                    else 0, axis=1)

In [20]:
# Of the responses where both 3.5 and 4 disagree with the human response, which ones both models DISagree with each other?
df_comp["FF_disagree"] = df_comp.apply(lambda x: 1 if (x["GPT 3.5_vs_H"] == 0 and
                                    x["GPT 4_vs_H"] == 0) and x["GPT 3.5"] != x["GPT 4"]
                                    else 0, axis=1)

In [21]:
df_comp

,Human,GPT 3.5,GPT 4,GPT 3.5_vs_H,GPT 4_vs_H,FF_agree,FF_disagree
0,A,C,A,0,1,0,0
1,C,C,C,1,1,0,0
2,A,A,A,1,1,0,0
3,C,C,C,1,1,0,0
4,C,C,C,1,1,0,0
5,B,B,B,1,1,0,0
6,A,A,A,1,1,0,0
7,B,A,B,0,1,0,0
8,A,A,A,1,1,0,0
9,D,D,D,1,1,0,0


In [22]:
total = len(df_comp)
print(total)

43


In [23]:
df_comp["GPT 3.5_vs_H"].sum()

32

In [24]:
df_comp["GPT 3.5_vs_H"].sum() / total

0.7441860465116279

In [25]:
df_comp["GPT 4_vs_H"].sum()

37

In [26]:
df_comp["GPT 4_vs_H"].sum() / total

0.8604651162790697

In [27]:
df_comp["FF_agree"].sum()

3

In [28]:
df_comp["FF_disagree"].sum()

2